---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

---

_**Use SageMaker's XGBoost to train a binary classification model and for a list of tumors in batch file, predict if each is malignant**_

_**It also shows how to use the input output joining / filter feature in Batch transform in details**_

---



## Background
This purpose of this notebook is to train a model using SageMaker's XGBoost and UCI's breast cancer diagnostic data set to illustrate at how to run batch inferences and how to use the Batch Transform I/O join feature. UCI's breast cancer diagnostic data set is available at https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29. The data set is also available on Kaggle at https://www.kaggle.com/uciml/breast-cancer-wisconsin-data. The purpose here is to use this data set to build a predictve model of whether a breast mass image indicates benign or malignant tumor. 



---

## Setup

Let's start by specifying:

* The SageMaker role arn used to give training and batch transform access to your data. The snippet below will use the same role used by your SageMaker notebook instance. Otherwise, specify the full ARN of a role with the SageMakerFullAccess policy attached.
* The S3 bucket that you want to use for training and storing model objects.

In [2]:
!pip3 install -U sagemaker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.4/388.4 kB 4.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 30.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 28.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 28.9 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 2.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 1.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 851.6 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.1/569.1 kB 5.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 2.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
import boto3
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name

bucket = sess.default_bucket()
prefix = "DEMO-breast-cancer-prediction-xgboost-highlevel"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


---
## Data sources

> Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

> Breast Cancer Wisconsin (Diagnostic) Data Set [https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)].

> _Also see:_ Breast Cancer Wisconsin (Diagnostic) Data Set [https://www.kaggle.com/uciml/breast-cancer-wisconsin-data].

## Data preparation


Let's download the data and save it in the local folder with the name data.csv and take a look at it.

In [4]:
import pandas as pd
import numpy as np

s3 = boto3.client("s3")

filename = "wdbc.csv"
s3.download_file(
    f"sagemaker-example-files-prod-{region}", "datasets/tabular/breast_cancer/wdbc.csv", filename
)
data = pd.read_csv(filename, header=None)

# specify columns extracted from wbdc.names
data.columns = [
    "id",
    "diagnosis",
    "radius_mean",
    "texture_mean",
    "perimeter_mean",
    "area_mean",
    "smoothness_mean",
    "compactness_mean",
    "concavity_mean",
    "concave points_mean",
    "symmetry_mean",
    "fractal_dimension_mean",
    "radius_se",
    "texture_se",
    "perimeter_se",
    "area_se",
    "smoothness_se",
    "compactness_se",
    "concavity_se",
    "concave points_se",
    "symmetry_se",
    "fractal_dimension_se",
    "radius_worst",
    "texture_worst",
    "perimeter_worst",
    "area_worst",
    "smoothness_worst",
    "compactness_worst",
    "concavity_worst",
    "concave points_worst",
    "symmetry_worst",
    "fractal_dimension_worst",
]

# save the data
data.to_csv("data.csv", sep=",", index=False)

data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
418,906024,B,12.700,12.17,80.88,495.0,0.08785,0.05794,0.023600,0.02402,...,13.65,16.92,88.12,566.9,0.1314,0.1607,0.09385,0.08224,0.2775,0.09464
205,879523,M,15.120,16.68,98.78,716.6,0.08876,0.09588,0.075500,0.04079,...,17.77,20.24,117.70,989.5,0.1491,0.3331,0.33270,0.12520,0.3415,0.09740
385,90291,M,14.600,23.29,93.97,664.7,0.08682,0.06636,0.083900,0.05271,...,15.79,31.71,102.20,758.2,0.1312,0.1581,0.26750,0.13590,0.2477,0.06836
379,9013838,M,11.080,18.83,73.30,361.6,0.12160,0.21540,0.168900,0.06367,...,13.24,32.82,91.76,508.1,0.2184,0.9379,0.84020,0.25240,0.4154,0.14030
73,859983,M,13.800,15.79,90.43,584.1,0.10070,0.12800,0.077890,0.05069,...,16.57,20.86,110.30,812.4,0.1411,0.3542,0.27790,0.13830,0.2589,0.10300
49,857156,B,13.490,22.30,86.91,561.0,0.08752,0.07698,0.047510,0.03384,...,15.15,31.82,99.00,698.8,0.1162,0.1711,0.22820,0.12820,0.2871,0.06917
89,861598,B,14.640,15.24,95.77,651.9,0.11320,0.13390,0.099660,0.07064,...,16.34,18.24,109.40,803.6,0.1277,0.3089,0.26040,0.13970,0.3151,0.08473
424,907145,B,9.742,19.12,61.93,289.7,0.10750,0.08333,0.008934,0.01967,...,11.21,23.17,71.79,380.9,0.1398,0.1352,0.02085,0.04589,0.3196,0.08009


#### Key observations:
* The data has 569 observations and 32 columns.
* The first field is the 'id' attribute that we will want to drop before batch inference and add to the final inference output next to the probability of malignancy.
* Second field, 'diagnosis', is an indicator of the actual diagnosis ('M' = Malignant; 'B' = Benign).
* There are 30 other numeric features that we will use for training and inferencing.

Let's replace the M/B diagnosis with a 1/0 boolean value. 

In [5]:
data["diagnosis"] = data["diagnosis"].apply(lambda x: ((x == "M")) + 0)
data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
344,89864002,0,11.710,15.45,75.03,420.3,0.11500,0.07281,0.040060,0.03250,...,13.060,18.16,84.16,516.4,0.14600,0.11150,0.10870,0.07864,0.2765,0.07806
406,905189,0,16.140,14.86,104.30,800.0,0.09495,0.08501,0.055000,0.04528,...,17.710,19.58,115.90,947.9,0.12060,0.17220,0.23100,0.11290,0.2778,0.07012
273,8910996,0,9.742,15.67,61.50,289.9,0.09037,0.04689,0.011030,0.01407,...,10.750,20.88,68.09,355.2,0.14670,0.09370,0.04043,0.05159,0.2841,0.08175
493,914101,0,12.460,12.83,78.83,477.3,0.07372,0.04043,0.007173,0.01149,...,13.190,16.36,83.24,534.0,0.09439,0.06477,0.01674,0.02680,0.2280,0.07028
472,911384,0,14.920,14.93,96.45,686.9,0.08098,0.08549,0.055390,0.03221,...,17.180,18.22,112.00,906.6,0.10650,0.27910,0.31510,0.11470,0.2688,0.08273
5,843786,1,12.450,15.70,82.57,477.1,0.12780,0.17000,0.157800,0.08089,...,15.470,23.75,103.40,741.6,0.17910,0.52490,0.53550,0.17410,0.3985,0.12440
64,85922302,1,12.680,23.84,82.69,499.0,0.11220,0.12620,0.112800,0.06873,...,17.090,33.47,111.80,888.3,0.18510,0.40610,0.40240,0.17160,0.3383,0.10310
568,92751,0,7.760,24.54,47.92,181.0,0.05263,0.04362,0.000000,0.00000,...,9.456,30.37,59.16,268.6,0.08996,0.06444,0.00000,0.00000,0.2871,0.07039


Let's split the data as follows: 80% for training, 10% for validation and let's set 10% aside for our batch inference job. In addition, let's drop the 'id' field on the training set and validation set as 'id' is not a training feature. For our batch set however, we keep the 'id' feature. We'll want to filter it out prior to running our inferences so that the input data features match the ones of training set and then ultimately, we'll want to join it with inference result. We are however dropping the diagnosis attribute for the batch set since this is what we'll try to predict.

In [6]:
# data split in three sets, training, validation and batch inference
rand_split = np.random.rand(len(data))
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
batch_list = rand_split >= 0.9

data_train = data[train_list].drop(["id"], axis=1)
data_val = data[val_list].drop(["id"], axis=1)
data_batch = data[batch_list].drop(["diagnosis"], axis=1)
data_batch_noID = data_batch.drop(["id"], axis=1)

Let's upload those data sets in S3

In [7]:
train_file = "train_data.csv"
data_train.to_csv(train_file, index=False, header=False)
sess.upload_data(train_file, key_prefix="{}/train".format(prefix))

validation_file = "validation_data.csv"
data_val.to_csv(validation_file, index=False, header=False)
sess.upload_data(validation_file, key_prefix="{}/validation".format(prefix))

batch_file = "batch_data.csv"
data_batch.to_csv(batch_file, index=False, header=False)
sess.upload_data(batch_file, key_prefix="{}/batch".format(prefix))

batch_file_noID = "batch_data_noID.csv"
data_batch_noID.to_csv(batch_file_noID, index=False, header=False)
sess.upload_data(batch_file_noID, key_prefix="{}/batch".format(prefix))

's3://sagemaker-us-east-1-002969758287/DEMO-breast-cancer-prediction-xgboost-highlevel/batch/batch_data_noID.csv'

---

## Training job and model creation

The below cell uses the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to kick off the training job using both our training set and validation set. Not that the objective is set to 'binary:logistic' which trains a model to output a probability between 0 and 1 (here the probability of a tumor being malignant).

In [8]:
%%time
from time import gmtime, strftime

job_name = "xgb-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
output_location = "s3://{}/{}/output/{}".format(bucket, prefix, job_name)
image = sagemaker.image_uris.retrieve(
    framework="xgboost", region=boto3.Session().region_name, version="1.7-1"
)

sm_estimator = sagemaker.estimator.Estimator(
    image,
    role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size=50,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sess,
)

sm_estimator.set_hyperparameters(
    objective="binary:logistic",
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    verbosity=0,
    num_round=100,
)

train_data = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train".format(bucket, prefix),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
validation_data = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validation".format(bucket, prefix),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
data_channels = {"train": train_data, "validation": validation_data}

# Start training by calling the fit method in the estimator
sm_estimator.fit(inputs=data_channels, job_name=job_name, logs=True)

INFO:sagemaker:Creating training-job with name: xgb-2024-10-25-22-29-14


2024-10-25 22:29:15 Starting - Starting the training job...
2024-10-25 22:29:32 Starting - Preparing the instances for training...
2024-10-25 22:30:19 Downloading - Downloading the training image......
2024-10-25 22:31:10 Training - Training image download completed. Training in progress..[2024-10-25 22:31:21.048 ip-10-0-139-186.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-10-25 22:31:21.073 ip-10-0-139-186.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-10-25:22:31:21:INFO] Imported framework sagemaker_xgboost_container.training
[2024-10-25:22:31:21:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2024-10-25:22:31:21:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-25:22:31:21:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-10-25:22:31:21:INFO] Determined 0 GPU(s) available on the instance.
[2024-10-25:22:31:21:INFO] Determined delimiter of C

---

## Batch Transform

In SageMaker Batch Transform, we introduced 3 new attributes - __input_filter__, __join_source__ and __output_filter__. In the below cell, we use the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to kick-off several Batch Transform jobs using different configurations of these 3 new attributes. Please refer to [this page](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform-data-processing.html) to learn more about how to use them.




#### 1. Create a transform job with the default configurations
Let's first skip these 3 new attributes and inspect the inference results. We'll use it as a baseline to compare to the results with data processing.

In [9]:
%%time

sm_transformer = sm_estimator.transformer(1, "ml.m4.xlarge")

# start a transform job
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file_noID
)  # use input data without ID column
sm_transformer.transform(input_location, content_type="text/csv", split_type="Line")
sm_transformer.wait()

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-10-25-22-34-14-595
INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-10-25-22-34-15-374


.............................................
[2024-10-25:22:41:48:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-25:22:41:48:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-25:22:41:48:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      return 404 "{}";
    }
  }
}
[2024-10-25 22:41:48 +0000] [18] [INFO

Let's inspect the output of the Batch Transform job in S3. It should show the list probabilities of tumors being malignant.

In [10]:
import re


def get_csv_output_from_s3(s3uri, batch_file):
    file_name = "{}.out".format(batch_file)
    match = re.match("s3://([^/]+)/(.*)", "{}/{}".format(s3uri, file_name))
    output_bucket, output_prefix = match.group(1), match.group(2)
    s3.download_file(output_bucket, output_prefix, file_name)
    return pd.read_csv(file_name, sep=",", header=None)

In [12]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file_noID)
output_df.head(8)

,0
0,0.884313
1,0.992090
2,0.005741
3,0.048952
4,0.005741
5,0.079594
6,0.006975
7,0.011179


#### 2. Join the input and the prediction results 
Now, let's associate the prediction results with their corresponding input records. We can also use the __input_filter__ to exclude the ID column easily and there's no need to have a separate file in S3.

* Set __input_filter__ to "$[1:]": indicates that we are excluding column 0 (the 'ID') before processing the inferences and keeping everything from column 1 to the last column (all the features or predictors)  
  
  
* Set __join_source__ to "Input": indicates our desire to join the input data with the inference results  

* Leave __output_filter__ to default ('$'), indicating that the joined input and inference results be will saved as output.

In [13]:
# content_type / accept and split_type / assemble_with are required to use IO joining feature
sm_transformer.assemble_with = "Line"
sm_transformer.accept = "text/csv"

# start a transform job
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file
)  # use input data with ID column cause InputFilter will filter it out
sm_transformer.transform(
    input_location,
    split_type="Line",
    content_type="text/csv",
    input_filter="$[1:]",
    join_source="Input",
)
sm_transformer.wait()

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-10-25-22-43-26-716


...........................................[2024-10-25:22:50:43:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-25:22:50:43:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-25:22:50:43:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      return 404 "{}";
    }
  }
}
[2024-10-25 22:50:43 +0000] [18] [INFO] S

Let's inspect the output of the Batch Transform job in S3. It should show the list of tumors identified by their original feature columns and their corresponding probabilities of being malignant.

In [14]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file)
output_df.head(8)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,844981,13.000,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.093530,0.2350,...,30.73,106.20,739.3,0.1703,0.54010,0.53900,0.20600,0.4378,0.10720,0.884313
1,84862001,16.130,20.68,108.10,798.8,0.11700,0.20220,0.17220,0.102800,0.2164,...,31.48,136.80,1315.0,0.1789,0.42330,0.47840,0.20730,0.3706,0.11420,0.992090
2,8510824,9.504,12.44,60.34,273.9,0.10240,0.06492,0.02956,0.020760,0.1815,...,15.66,65.13,314.9,0.1324,0.11480,0.08867,0.06227,0.2450,0.07773,0.005741
3,855167,13.440,21.58,86.18,563.0,0.08162,0.06031,0.03110,0.020310,0.1784,...,30.25,102.50,787.9,0.1094,0.20430,0.20850,0.11120,0.2994,0.07146,0.048952
4,85713702,8.196,16.84,51.71,201.9,0.08600,0.05943,0.01588,0.005917,0.1769,...,21.96,57.26,242.2,0.1297,0.13570,0.06880,0.02564,0.3105,0.07409,0.005741
5,857156,13.490,22.30,86.91,561.0,0.08752,0.07698,0.04751,0.033840,0.1809,...,31.82,99.00,698.8,0.1162,0.17110,0.22820,0.12820,0.2871,0.06917,0.079594
6,85759902,11.520,18.75,73.34,409.0,0.09524,0.05473,0.03036,0.022780,0.1920,...,22.47,81.81,506.2,0.1249,0.08720,0.09076,0.06316,0.3306,0.07036,0.006975
7,858970,10.170,14.88,64.55,311.9,0.11340,0.08061,0.01084,0.012900,0.2743,...,17.45,69.86,368.6,0.1275,0.09866,0.02168,0.02579,0.3557,0.08020,0.011179


#### 3. Update the output filter to keep only ID and prediction results
Let's change __output_filter__ to "$[0,-1]", indicating that when presenting the output, we only want to keep column 0 (the 'ID') and the last column (the inference result i.e. the probability of a given tumor to be malignant)

In [15]:
# start another transform job
sm_transformer.transform(
    input_location,
    split_type="Line",
    content_type="text/csv",
    input_filter="$[1:]",
    join_source="Input",
    output_filter="$[0,-1]",
)
sm_transformer.wait()

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-10-25-22-57-34-970


.........................................[2024-10-25:23:04:22:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-25:23:04:22:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-25:23:04:22:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
[2024-10-25:23:04:22:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-25:23:04:22:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-25:23:04:22:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|

Now, let's inspect the output of the Batch Transform job in S3 again. It should show 2 columns: the ID and their corresponding probabilities of being malignant.

In [16]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file)
output_df.head(8)

,0,1
0,844981,0.884313
1,84862001,0.992090
2,8510824,0.005741
3,855167,0.048952
4,85713702,0.005741
5,857156,0.079594
6,85759902,0.006975
7,858970,0.011179


create_model(role=role, image_uri=XGBOOST_IMAGE)In summary, we can use newly introduced 3 attributes - __input_filter__, __join_source__, __output_filter__ to 
1. Filter / select useful features from the input dataset. e.g. exclude ID columns.
2. Associate the prediction results with their corresponding input records.
3. Filter the original or joined results before saving to S3. e.g. keep ID and probability columns only.

## Upload the Sagemaker Model created during our training job to the Sagemaker Model Registry

In [18]:
sagemaker = boto3.client("sagemaker")

model_name = "xgb-2024-10-25-22-29-14"
print(model_name)


info = sagemaker.describe_training_job(TrainingJobName=model_name)
model_data = info["ModelArtifacts"]["S3ModelArtifacts"]

primary_container = {"Image": image, "ModelDataUrl": model_data}

# Save our model to the Sagemaker Model Registry
create_model_response = sagemaker.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

xgb-2024-10-25-22-29-14
arn:aws:sagemaker:us-east-1:002969758287:model/xgb-2024-10-25-22-29-14


In [19]:
# Inspect Training Job Details
info

{'TrainingJobName': 'xgb-2024-10-25-22-29-14',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:002969758287:training-job/xgb-2024-10-25-22-29-14',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-002969758287/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-10-25-22-29-14/xgb-2024-10-25-22-29-14/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'eta': '0.2',
  'gamma': '4',
  'max_depth': '5',
  'min_child_weight': '6',
  'num_round': '100',
  'objective': 'binary:logistic',
  'subsample': '0.8',
  'verbosity': '0'},
 'AlgorithmSpecification': {'TrainingImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1',
  'TrainingInputMode': 'File',
  'MetricDefinitions': [{'Name': 'train:mae',
    'Regex': '.*\\[[0-9]+\\].*#011train-mae:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
   {'Name': 'validation:aucpr',
    'Regex': '.*\\[[0-9]+\\].*#011validation-aucpr:([-+]?[0-9]*\

In [20]:
# Create Endpoint Configuration


# Create an endpoint config name. Here we create one based on the date  
# so it we can search endpoints based on creation time.
endpoint_config_name = 'lab4-1-endpoint-config' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())                            
                            
instance_type = 'ml.m5.xlarge'

endpoint_config_response = sagemaker.create_endpoint_config(
    EndpointConfigName=endpoint_config_name, # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name, 
            "InstanceType": instance_type, # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")


Created EndpointConfig: arn:aws:sagemaker:us-east-1:002969758287:endpoint-config/lab4-1-endpoint-config2024-10-25-23-11-05


In [23]:
# Deploy our model to real-time endpoint

endpoint_name = 'lab4-1-endpoint' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())                            


create_endpoint_response = sagemaker.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name) 

In [26]:
# Wait for endpoint to spin up
from time import sleep

sagemaker.describe_endpoint(EndpointName=endpoint_name)

while True:
    print("Getting Job Status")
    res = sagemaker.describe_endpoint(EndpointName=endpoint_name)
    state = res["EndpointStatus"]
    
    if state == "InService":
        print("Endpoint in Service")
        break
    elif state == "Creating":
        print("Endpoint still creating...")
        sleep(60)
    else:
        print("Endpoint Creation Error - Check Sagemaker Console")
        break

Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint in Service


In [27]:
# Invoke Endpoint

sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

response = sagemaker_runtime.invoke_endpoint(
                            EndpointName=endpoint_name,
                            ContentType='text/csv',
                            Body=data_batch_noID.to_csv(header=None, index=False).strip('\n').split('\n')[0]
                            )
print(response['Body'].read().decode('utf-8'))

0.8843128085136414



In [28]:
# Examine Response Body

response

{'ResponseMetadata': {'RequestId': '50fed05c-f825-45c9-86a3-f0bba5d0913f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '50fed05c-f825-45c9-86a3-f0bba5d0913f',
   'x-amzn-invoked-production-variant': 'variant1',
   'date': 'Fri, 25 Oct 2024 23:17:41 GMT',
   'content-type': 'text/csv; charset=utf-8',
   'content-length': '19',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'variant1',
 'Body': <botocore.response.StreamingBody at 0x7ff6eb379c90>}

# Assignment 4.1 

Name: Ryan Laxamana

# Part 1: Set Up Model Group

In [32]:
import time
import boto3
from sagemaker import get_execution_role, Session

# Set up session and client
region = boto3.Session().region_name
role = get_execution_role()
sagemaker_session = Session(boto3.Session(region_name=region))
sm_client = boto3.client('sagemaker', region_name=region)

# Define the model package group name and description
model_package_group_name = f"breast-cancer-tumor-detection-{int(time.time())}"
model_package_group_input_dict = {
    "ModelPackageGroupName": model_package_group_name,
    "ModelPackageGroupDescription": "Sample model package group for classifying tumors for breast cancer",
}

# Create model package group in SageMaker Model Registry
create_model_package_group_response = sm_client.create_model_package_group(
    **model_package_group_input_dict
)
print(f"ModelPackageGroup Arn : {create_model_package_group_response['ModelPackageGroupArn']}")


ModelPackageGroup Arn : arn:aws:sagemaker:us-east-1:002969758287:model-package-group/breast-cancer-tumor-detection-1729898725


# Part 2: Set Up Model Package

In [39]:
# List model packages in a specific model package group
model_package_group_name = 'breast-cancer-tumor-detection-1729898725'
response = sm_client.list_model_packages(ModelPackageGroupName=model_package_group_name)

# Print out model package details
for package in response['ModelPackageSummaryList']:
    print("Model Package ARN:", package['ModelPackageArn'])
    print("Model Package Name:", package['ModelPackageGroupName'])


Model Package ARN: arn:aws:sagemaker:us-east-1:002969758287:model-package/breast-cancer-tumor-detection-1729898725/1
Model Package Name: breast-cancer-tumor-detection-1729898725


In [40]:
import boto3

# Initialize SageMaker client
sm_client = boto3.client('sagemaker', region_name='us-east-1')

# Use the correct Model Package ARN
model_package_arn = 'arn:aws:sagemaker:us-east-1:002969758287:model-package/breast-cancer-tumor-detection-1729898725/1'

# Describe the model package using the ARN
describe_response = sm_client.describe_model_package(ModelPackageName=model_package_arn)

# Print the output
print(describe_response)


{'ModelPackageGroupName': 'breast-cancer-tumor-detection-1729898725', 'ModelPackageVersion': 1, 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:002969758287:model-package/breast-cancer-tumor-detection-1729898725/1', 'ModelPackageDescription': 'Model to determine if a breast tumor is malignant or benign based on several medical characteristics', 'CreationTime': datetime.datetime(2024, 10, 26, 0, 7, 25, 759000, tzinfo=tzlocal()), 'InferenceSpecification': {'Containers': [{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1', 'ImageDigest': 'sha256:b4a5f397d97c886f58dc45f9cae22d3479eb9dad7b06015844fdc745955df41c', 'ModelDataUrl': 's3://sagemaker-us-east-1-002969758287/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-10-25-22-29-14/xgb-2024-10-25-22-29-14/output/model.tar.gz', 'Framework': 'XGBOOST', 'NearestModelName': 'Breast cancer prediction xgboost'}], 'SupportedRealtimeInferenceInstanceTypes': ['ml.t2.medium'], 'SupportedContentTypes': ['text

# Part 3 Write the Model Card

In [44]:
# Initialize the SageMaker client
client = boto3.client('sagemaker', region_name='us-east-1')  

# Define Model Card Name and Version
model_card_name = 'breast-cancer-tumor-detection-1729898725'
model_card_version = 1

# Describe the model card
try:
    response = client.describe_model_card(
        ModelCardName=model_card_name,
        ModelCardVersion=model_card_version
    )
    # Print the response
    print(response)

except client.exceptions.ResourceNotFoundException:
    print("The specified model card or version does not exist.")
except Exception as e:
    print(f"An error occurred: {e}")

{'ModelCardArn': 'arn:aws:sagemaker:us-east-1:002969758287:model-card/breast-cancer-tumor-detection-1729898725', 'ModelCardName': 'breast-cancer-tumor-detection-1729898725', 'ModelCardVersion': 1, 'Content': '{"model_overview":{"model_name":"breast-cancer-tumor-detection-1729898725","model_id":"breast-cancer-tumor-detection-1729898725 v1","model_artifact":["s3://sagemaker-us-east-1-002969758287/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-10-25-22-29-14/xgb-2024-10-25-22-29-14/output/model.tar.gz"],"model_version":1,"problem_type":"Binary classification","algorithm_type":"Linear regression","model_description":"Model to determine if a breast tumor is malignant or benign based on several medical characteristics","model_creator":"RLaxamana_AAI540","model_owner":"RLaxamana","inference_environment":{"container_image":["683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1"]}},"intended_uses":{"intended_uses":"Classify breast tumors"},"training_details":{"o

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ca-central-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/sa-east-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-3/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-central-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-north-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-south-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)
